### uw2

In [ ]:
import underworld as uw
from underworld import function as fn
import underworld.visualisation as vis
import math
import numpy as np
import os

import matplotlib.pyplot as plt
import numpy as np


In [ ]:
def load_mesh_uw2(output_path,meshCopy,step):
    meshfile = output_path+'mesh.'+ str(step).zfill(5) +'.h5'
    meshCopy.load(meshfile)
    return meshCopy

def load_swarm_uw2(output_path,meshCopy,step,swarm_name):
    swarmCopy = uw.swarm.Swarm(mesh=meshCopy,particleEscape=True)
    swarmfile = output_path+swarm_name + str(step).zfill(5) +'.h5'
    swarmCopy.load(swarmfile)
    return swarmCopy

def load_mesh_var_uw2(output_path,meshCopy,step,var_name,nodeDofCount):
    varField = meshCopy.add_variable(nodeDofCount=nodeDofCount)
    varfile = output_path+var_name+ str(step).zfill(5) +'.h5'
    varField.load(varfile)
    return varField

def load_swarm_uwg(output_path,meshCopy,step,swarm_name):
    swarmCopy = uw.swarm.Swarm(mesh=meshCopy,particleEscape=True)
    swarmfile = output_path+swarm_name+ str(step)+".h5"
    swarmCopy.load(swarmfile)
    return swarmCopy

def load_mesh_var_uwg(output_path,meshCopy,step,var_name,nodeDofCount):
    varField = meshCopy.add_variable(nodeDofCount=nodeDofCount)
    varfile = output_path+var_name+ str(step) +'.h5'
    varField.load(varfile)
    return varField

def load_swarm_var_uwg(output_path,swarmCopy,step,swarm_name):
    materialField = swarmCopy.add_variable("int",1)
    filename = output_path+"materialField-"+ str(step)+".h5"
    materialField.load(filename)

### uw3

In [ ]:
import petsc4py
from petsc4py import PETSc

import underworld3 as uw
from underworld3.systems import Stokes
from underworld3 import function
import numpy as np

import matplotlib.pyplot as plt
import pyvista as pv
import glob

In [ ]:
def load_swarm_uw3(output_path,load_step,swarmName):
    swarmFilename =  f"{output_path}{swarmName}_{load_step:05d}.xmf"
    swarmData = pv.XdmfReader(swarmFilename).read()
    particleCoordinates = swarmData.points
    return particleCoordinates

def load_mesh_var_uw3(output_path,load_step,var_name):
    pv_mesh_data = pv.XdmfReader(output_path + f'step_{load_step:05d}.xdmf').read()
    var_d =  pv_mesh_data[var_name]
    return var_d

In [ ]:
def load_depth_uw3(fdir,maxstep,dstep):
    depth_l = []
    depth_r = []
    time_l = []
    for step in range(0,maxstep+1,dstep):
        load_step = f"{step:05d}" 
        #swarmCopy= uw.swarm.Swarm(mesh)
        swarmName  = "surfaceSwarm"
        swarmFilename =  f"{fdir}{swarmName}_{load_step}.xmf"
        swarmData = pv.XdmfReader(swarmFilename).read()
        xcoord,ycoord= swarmData.points[:,0],swarmData.points[:,1]
        
        pv_mesh_data = pv.XdmfReader(fdir + f'step_{load_step}.xdmf').read()
        time_d =  pv_mesh_data['time_time'][0]

        depth_l.append(ycoord[0])   
        depth_r.append(ycoord[1])  
        time_l.append(time_d)  
    return np.array(depth_l),np.array(depth_r),time_l

In [ ]:
from petsc4py import PETSc
comm = PETSc.COMM_WORLD


def find_topwall_mid(outputPath,load_step):
    filename = outputPath + f'mesh._step_{load_step:05d}.h5'
    viewer = PETSc.ViewerHDF5().create(filename, "r", comm=comm)
    h5plex = PETSc.DMPlex().create(comm=comm)
    sf0 = h5plex.topologyLoad(viewer)
    h5plex.coordinatesLoad(viewer, sf0)
    h5plex.labelsLoad(viewer, sf0)

    cdim = 2
    dm = h5plex
    topwall = petsc_dm_find_labeled_points_local(dm,"Top")
    coord_vec = dm.getCoordinatesLocal()
    coords = coord_vec.array.reshape(-1, cdim)
    
    def find_ind(value):
        topwall_x = coords[topwall,0]
        idx = np.abs(topwall_x-value).argmin()
        return idx
    idx = find_ind(0)
    topwallmid_Ind = topwall[idx] 
    return coords[topwallmid_Ind,1]